# Top 50 US Business Schools

# 1. Introduction

A friend of mine is working for a company that has a company-sponsored study program, where its employees can obtain an MBA degree. 
My friend recently applied for the program and received an approval from the company. 
Now he is searching for a business school to apply for, and asked me for advice.

The company’s policy only requires that the business school for the program should be chosen from the top 50 business schools in the US, 
and my friend told me that he is indifferent among the top 50 schools because all of them are great schools and that he would like to decide based on what is available around the campus of each business school/university. 

Therefore, in this project, I will compare the top 50 business schools in the US based on information about the neighborhood of each campus by using the cluster analysis. 

The result will be used to guide my friend’s decision on which school(s) to apply for.

# 2. Data

For this project, I used the list of United States graduate business school rankings, which is available on Wikipedia.

URL = https://en.wikipedia.org/wiki/List_of_United_States_graduate_business_school_rankings

Specifically, I use the ranking by US News in 2019 to obtain the list of the top 50 business schools because:

(a) that is most recent, and (b) the ranking focuses on US business schools. 

Then, the geograohical information (latitude and longitude) of each university/school was obtained by using Nominatim, and the information was subsequently used for exploring the neighborhood with the Foursquare location data.

Finally, the neghborhood data of each university/school was used to cluster the 50 schools to gain insigths on which schools have smilar neighborhood.


# 3. Method

## 3-1. Import data from Wikipedia

First, I imported the US business school ranking from the wikipedia page mentioned in the Data section and stored in in a dataframe using BeutifulSoup.

The data set contains 119 rows and 4 columns, which are 'Business School', 'University', 'Location (City and State)' and 'US News rankings'

In [10]:
import numpy as np
import pandas as pd
import urllib.request

url = 'https://en.wikipedia.org/wiki/List_of_United_States_graduate_business_school_rankings'
page = urllib.request.urlopen(url)

from bs4 import BeautifulSoup

soup = BeautifulSoup(page)
right_table = soup.find('table', class_ = 'wikitable sortable')

A=[]
B=[]
C=[]
D=[]
E=[]
F=[]
G=[]
H=[]
I=[]
J=[]
K=[]
L=[]

for row in right_table.findAll("tr"):
    cells = row.findAll('td')
    states = row.findAll('th') 
    if len(cells)==12: 
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
        D.append(cells[3].find(text=True))
        E.append(cells[4].find(text=True))
        F.append(cells[5].find(text=True))
        G.append(cells[6].find(text=True))
        H.append(cells[7].find(text=True))
        I.append(cells[8].find(text=True))
        J.append(cells[9].find(text=True))
        K.append(cells[10].find(text=True))
        L.append(cells[11].find(text=True))

df_mba = pd.DataFrame(list(zip(A, B, C, D)),
                columns = ['BusinessSchool', 'University', 'Location', 'USN2019'])
df_mba.head()




,BusinessSchool,University,Location,USN2019
0,A. Gary Anderson Graduate School of Management,University of California Riverside,"California, Riverside",91
1,Argyros School of Business and Economics,Chapman University,"California, Orange",85
2,Atkinson Graduate School of Management,Willamette University,"Oregon, Salem",99-131
3,Babcock Graduate School of Management,Wake Forest University,"North Carolina, Winston-Salem",NR
4,Bauer College of Business,University of Houston,"Texas, Houston",95


In [11]:
df_mba.shape

(119, 4)

In [12]:
df_mba.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119 entries, 0 to 118
Data columns (total 4 columns):
BusinessSchool    119 non-null object
University        119 non-null object
Location          119 non-null object
USN2019           119 non-null object
dtypes: object(4)
memory usage: 3.8+ KB


## 3-2. Data Clean-up

Next, the data was cleaned up so that it can be sorted based on the ranking and slect top 100 schools.

First, the rows containing "99-131" or "NR" were removed since the focus of this project is on the top 50 business schools.

In [21]:
df_mba2 = df_mba.loc[(df_mba['USN2019'] != '99-131\n') & 
                          (df_mba['USN2019'] != 'NR\n')]
df_mba2.replace('\n','')

df_mba2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 118
Data columns (total 4 columns):
BusinessSchool    92 non-null object
University        92 non-null object
Location          92 non-null object
USN2019           92 non-null object
dtypes: object(4)
memory usage: 3.6+ KB


The resulting dataframe contains 92 rows.

Next, the data type of the 'USN2019' column was changed from 'object' to 'int' so that the business schools can be sorted based on the ranking.

Also, the data type of the 'University' column was changed from 'object' to 'str'.

In [22]:
df_mba3 = df_mba2
df_mba3['USN2019'] = df_mba2['USN2019'].astype('int')
df_mba3['University'] = df_mba2['University'].astype('str')
df_mba3.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,BusinessSchool,University,Location,USN2019
0,A. Gary Anderson Graduate School of Management,University of California Riverside,"California, Riverside",91
1,Argyros School of Business and Economics,Chapman University,"California, Orange",85
4,Bauer College of Business,University of Houston,"Texas, Houston",95
5,Bennett S. LeBow College of Business,Drexel University,"Pennsylvania, Philadelphia",84
6,Binghamton University School of Management,Binghamton University,"New York, Binghamton",89


And then, the labels 'University of California, Los Angeles' and 'University of wisconsin at Madison' were changed to 'University of California Los Angeles', 'University of Wisconsin-Madison', respectively, so that these labels can be used as query for Nominatim (the original labels did not work with Nominatim).

In [23]:
df_mba4 = df_mba3.replace(['University of California, Los Angeles', 'University of Wisconsin at Madison'], ['University of California Los Angeles', 'University of Wisconsin-Madison'])
df_mba4.head()

,BusinessSchool,University,Location,USN2019
0,A. Gary Anderson Graduate School of Management,University of California Riverside,"California, Riverside",91
1,Argyros School of Business and Economics,Chapman University,"California, Orange",85
4,Bauer College of Business,University of Houston,"Texas, Houston",95
5,Bennett S. LeBow College of Business,Drexel University,"Pennsylvania, Philadelphia",84
6,Binghamton University School of Management,Binghamton University,"New York, Binghamton",89


Finally, a new dataframe was created containing the top 49 business schools and sorted based on the US News 2019 ranking.

In [24]:
df_mba5 = df_mba4.sort_values(by = ['USN2019'])
df_mba6 = df_mba5.loc[df_mba5['USN2019'] <50]
df_mba7 = df_mba6.reset_index(drop = True)
df_mba7

,BusinessSchool,University,Location,USN2019
0,The Wharton School,University of Pennsylvania,"Pennsylvania, Philadelphia",1
1,Stanford Graduate School of Business,Stanford University,"California, Stanford",2
2,Harvard Business School,Harvard University,"Massachusetts, Boston",3
3,MIT Sloan School of Management,Massachusetts Institute of Technology,"Massachusetts, Cambridge",3
4,Booth School of Business,University of Chicago,"Illinois, Chicago",3
5,Kellogg School of Management,Northwestern University,"Illinois, Evanston",6
6,Columbia Business School,Columbia University,"New York, New York City",6
7,Haas School of Business,"University of California, Berkeley","California, Berkeley",6
8,Yale School of Management,Yale University,"Connecticut, New Haven",9
9,Ross School of Business,University of Michigan,"Michigan, Ann Arbor",10


In [25]:
df_mba7.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 4 columns):
BusinessSchool    49 non-null object
University        49 non-null object
Location          49 non-null object
USN2019           49 non-null int64
dtypes: int64(1), object(3)
memory usage: 1.6+ KB


## 3-3. Plot the top business schools onto a map of USA 

In this section, the location of each of the top school/university was plotted onto a map of USA using Nominatim and Folium to get an insight of where each of the top business schools is located in the US.

First, libraries necessary for the analysis wer imported.  

In [26]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1e             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.11.28         |   py36h9f0ad1d_1         149 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    ------------------------------------------------------------
                       

Next, the map object of USA was created by using the geographical information obtained from Nominatim.

In [27]:
address = 'United States of America'

geolocator = Nominatim(user_agent="usa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

The geograpical coordinate are 39.7837304, -100.4458825.


In [28]:
map_usa = folium.Map(location=[latitude, longitude], zoom_start=4)

map_usa

To plot the locations of the top US business schools onto the map, I first created a list of the 49 universities from the dataframe created in the previous section.  

In [29]:
university = df_mba7['University'].values.tolist()
university

['University of Pennsylvania',
 'Stanford University',
 'Harvard University',
 'Massachusetts Institute of Technology',
 'University of Chicago',
 'Northwestern University',
 'Columbia University',
 'University of California, Berkeley',
 'Yale University',
 'University of Michigan',
 'Duke University',
 'Dartmouth College',
 'University of Virginia',
 'New York University',
 'Cornell University',
 'University of California Los Angeles',
 'University of Southern California',
 'Carnegie Mellon University',
 'University of North Carolina, Chapel Hill',
 'University of Texas at Austin',
 'Emory University',
 'Indiana University',
 'University of Washington',
 'Georgetown University',
 'University of Florida',
 'Rice University',
 'Washington University in St. Louis',
 'University of Notre Dame',
 'Vanderbilt University',
 'Georgia Institute of Technology',
 'Ohio State University',
 'Brigham Young University',
 'Arizona State University',
 'Penn State University',
 'University of Wisconsin

Then, the list was used to obtain geographical information (i.e. latitude and longitude) of each school.

In [30]:
from folium import plugins

mba_lat = []
mba_lng = []

for univ in university:
    g = Nominatim(user_agent="mba_explorer")
    loc = g.geocode(univ)
    if loc is not None:
        lat = loc.latitude
        lng = loc.longitude
        mba_lat.append(lat)
        mba_lng.append(lng)
    else:
        lat = 0
        lng = 0
        mba_lat.append(lat)
        mba_lng.append(lng)

df_mba_loc = pd.DataFrame(university, columns=['University'])
df_mba_loc['Latitude'] = mba_lat
df_mba_loc['Longitude'] = mba_lng
df_mba_loc

,University,Latitude,Longitude
0,University of Pennsylvania,39.949338,-75.189644
1,Stanford University,37.431314,-122.169365
2,Harvard University,42.367909,-71.126782
3,Massachusetts Institute of Technology,42.358396,-71.095678
4,University of Chicago,41.791351,-87.600843
5,Northwestern University,42.055116,-87.675811
6,Columbia University,40.807949,-73.961797
7,"University of California, Berkeley",37.870946,-122.266399
8,Yale University,41.257131,-72.989670
9,University of Michigan,42.294214,-83.710039


In [31]:
df_mba8 = pd.merge(df_mba7, df_mba_loc, on='University', how='left')
df_mba8

,BusinessSchool,University,Location,USN2019,Latitude,Longitude
0,The Wharton School,University of Pennsylvania,"Pennsylvania, Philadelphia",1,39.949338,-75.189644
1,Stanford Graduate School of Business,Stanford University,"California, Stanford",2,37.431314,-122.169365
2,Harvard Business School,Harvard University,"Massachusetts, Boston",3,42.367909,-71.126782
3,MIT Sloan School of Management,Massachusetts Institute of Technology,"Massachusetts, Cambridge",3,42.358396,-71.095678
4,Booth School of Business,University of Chicago,"Illinois, Chicago",3,41.791351,-87.600843
5,Kellogg School of Management,Northwestern University,"Illinois, Evanston",6,42.055116,-87.675811
6,Columbia Business School,Columbia University,"New York, New York City",6,40.807949,-73.961797
7,Haas School of Business,"University of California, Berkeley","California, Berkeley",6,37.870946,-122.266399
8,Yale School of Management,Yale University,"Connecticut, New Haven",9,41.257131,-72.989670
9,Ross School of Business,University of Michigan,"Michigan, Ann Arbor",10,42.294214,-83.710039


Finally, the location of each school/university was plotted onto the US map.

In [32]:
for label, lat, lng in zip(df_mba8['University'], df_mba8['Latitude'], df_mba8['Longitude']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_usa)
                           
map_usa

From this map, we can conclude that the distribution of the top 49 business schools is heavily biased towords the eastern side of the USA, particularly in the northeast region. 

## 3-4. Compare the neighborhood of the top business schools

In this section, I compared the top 49 business schools in the USA based on their neighborhood.

Specifically, information provided by Foursquare API was used on up to 200 venues that are in each neighborhood within a radius of 2000 meters from each university's campus.

In [33]:
CLIENT_ID = '5HJPMDR1CNEJ5LZA5MSYIJWHSA0IFAOOEM3423LH5IC12Y25' # your Foursquare ID
CLIENT_SECRET = 'QRLF5UCVYIRBP4AJ3XOW4QR5PHJR30TASNXMJ3D3XIYM5CGT' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5HJPMDR1CNEJ5LZA5MSYIJWHSA0IFAOOEM3423LH5IC12Y25
CLIENT_SECRET:QRLF5UCVYIRBP4AJ3XOW4QR5PHJR30TASNXMJ3D3XIYM5CGT


In [34]:
LIMIT = 200 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius

def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['University', 
                  'University_Latitude', 
                  'University_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)

In [35]:
mba_venues = getNearbyVenues(names = df_mba8['University'],
                               latitudes = df_mba8['Latitude'],
                               longitudes = df_mba8['Longitude']
                               )

University of Pennsylvania
Stanford University
Harvard University
Massachusetts Institute of Technology
University of Chicago
Northwestern University
Columbia University
University of California, Berkeley
Yale University
University of Michigan
Duke University
Dartmouth College
University of Virginia
New York University
Cornell University
University of California Los Angeles
University of Southern California
Carnegie Mellon University
University of North Carolina, Chapel Hill
University of Texas at Austin
Emory University
Indiana University
University of Washington
Georgetown University
University of Florida
Rice University
Washington University in St. Louis
University of Notre Dame
Vanderbilt University
Georgia Institute of Technology
Ohio State University
Brigham Young University
Arizona State University
Penn State University
University of Wisconsin-Madison
University of Minnesota
University of Georgia
University of Texas at Dallas
Michigan State University
Texas A&M University
Univer

In [36]:
print(mba_venues.shape)
mba_venues.head()

(4532, 7)


,University,University_Latitude,University_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,University of Pennsylvania,39.949338,-75.189644,University of Pennsylvania Museum of Archaeolo...,39.949525,-75.191418,History Museum
1,University of Pennsylvania,39.949338,-75.189644,World Cafe Live,39.952114,-75.185271,Music Venue
2,University of Pennsylvania,39.949338,-75.189644,Penn Park,39.951241,-75.185166,Park
3,University of Pennsylvania,39.949338,-75.189644,Schuylkill Banks Boardwalk,39.948405,-75.184236,Trail
4,University of Pennsylvania,39.949338,-75.189644,Zavino,39.953930,-75.188965,Italian Restaurant


In [37]:
mba_venues.groupby('University').count()

,University_Latitude,University_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
University,,,,,,
Arizona State University,100,100,100,100,100,100
Boston College,100,100,100,100,100,100
Brigham Young University,100,100,100,100,100,100
Carnegie Mellon University,100,100,100,100,100,100
Columbia University,100,100,100,100,100,100
Cornell University,61,61,61,61,61,61
Dartmouth College,52,52,52,52,52,52
Duke University,67,67,67,67,67,67
Emory University,99,99,99,99,99,99


In [38]:
print('There are {} uniques categories.'.format(len(mba_venues['Venue_Category'].unique())))

There are 360 uniques categories.


From the above analysis, 360 unique venues were identified for the top 49 business schools in the US.

## 3-5. Cluster analysis

In this section, the cluster analysis was performed for the top 49 business schools based on the information on thier neighborhood.

First, one hot encoding was done for clustering analysis.

In [39]:
# one hot encoding
mba_onehot = pd.get_dummies(mba_venues[['Venue_Category']], prefix="", prefix_sep="")

# add university column back to dataframe
mba_onehot['University'] = mba_venues['University'] 

# move university column to the first column
fixed_columns = [mba_onehot.columns[-1]] + list(mba_onehot.columns[:-1])
mba_onehot = mba_onehot[fixed_columns]

mba_onehot.head()

,University,ATM,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,University of Pennsylvania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,University of Pennsylvania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,University of Pennsylvania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,University of Pennsylvania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,University of Pennsylvania,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
mba_onehot.shape

(4532, 361)

In [41]:
mba_grouped = mba_onehot.groupby('University').mean().reset_index()
mba_grouped

,University,ATM,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Arizona State University,0.00,0.00,0.00,0.030000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.010000,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.00
1,Boston College,0.00,0.00,0.00,0.030000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Brigham Young University,0.02,0.00,0.00,0.010000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,Carnegie Mellon University,0.00,0.00,0.00,0.030000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000,0.00
4,Columbia University,0.00,0.00,0.02,0.040000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.010000,0.010000,0.010000,0.00,0.000000,0.000000,0.020000,0.000000,0.00
5,Cornell University,0.00,0.00,0.00,0.016393,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.032787,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Dartmouth College,0.00,0.00,0.00,0.076923,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,Duke University,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.014925,0.014925,0.00
8,Emory University,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.010101,0.000000,0.00,0.000000,0.010101,0.010101,0.000000,0.00
9,Georgetown University,0.00,0.00,0.00,0.050000,0.00,0.00,0.00,0.00,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.030000,0.000000,0.00


Next, the top five venues in each neighborhood was listed to see if the analysis was done as intended.

In [42]:
num_top_venues = 5

for univ in mba_grouped['University']:
    print("----"+univ+"----")
    temp = mba_grouped[mba_grouped['University'] == univ].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arizona State University----
                venue  freq
0         Pizza Place  0.09
1         Coffee Shop  0.08
2  Mexican Restaurant  0.04
3               Hotel  0.03
4      Sandwich Place  0.03


----Boston College----
                venue  freq
0         Pizza Place  0.07
1                Café  0.05
2  Mexican Restaurant  0.04
3         Coffee Shop  0.04
4      Ice Cream Shop  0.04


----Brigham Young University----
                  venue  freq
0        Sandwich Place  0.09
1  Fast Food Restaurant  0.07
2           Pizza Place  0.06
3    Mexican Restaurant  0.05
4        Ice Cream Shop  0.04


----Carnegie Mellon University----
                venue  freq
0         Coffee Shop  0.06
1      Sandwich Place  0.05
2  Mexican Restaurant  0.04
3               Hotel  0.03
4    Asian Restaurant  0.03


----Columbia University----
                 venue  freq
0                 Park  0.09
1          Coffee Shop  0.07
2   Italian Restaurant  0.05
3   Seafood Restaurant  0.04
4  American

Then, I made a dataframe listing the top 10 venues in each neighborhood by defining a new function, 'return_most_common_venues'.

In [43]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['University']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
mba_venues_sorted = pd.DataFrame(columns=columns)
mba_venues_sorted['University'] = mba_grouped['University']

for ind in np.arange(mba_grouped.shape[0]):
    mba_venues_sorted.iloc[ind, 1:] = return_most_common_venues(mba_grouped.iloc[ind, :], num_top_venues)

mba_venues_sorted.head()

,University,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arizona State University,Pizza Place,Coffee Shop,Mexican Restaurant,Hotel,Burger Joint,Sandwich Place,Bar,American Restaurant,Breakfast Spot,Theater
1,Boston College,Pizza Place,Café,Coffee Shop,Gym,Ice Cream Shop,Mexican Restaurant,Sushi Restaurant,Italian Restaurant,American Restaurant,Chinese Restaurant
2,Brigham Young University,Sandwich Place,Fast Food Restaurant,Pizza Place,Mexican Restaurant,Ice Cream Shop,Bakery,Burger Joint,ATM,Discount Store,Storage Facility
3,Carnegie Mellon University,Coffee Shop,Sandwich Place,Mexican Restaurant,American Restaurant,Asian Restaurant,Hotel,New American Restaurant,Indian Restaurant,Ice Cream Shop,Turkish Restaurant
4,Columbia University,Park,Coffee Shop,Italian Restaurant,American Restaurant,Seafood Restaurant,Bakery,Grocery Store,Bookstore,Chinese Restaurant,Bagel Shop


Finally, the top 49 business schools were classified into 5 clusters by using the k-means clustering method. 

In [45]:
# set number of clusters
kclusters = 5

mba_grouped_clustering = mba_grouped.drop('University', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mba_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 1, 1, 2, 2, 2, 2, 1, 1, 2], dtype=int32)

In [46]:
#add clustering labels
mba_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mba_merged = df_mba8

# merge mba_grouped with df_mba8 to add latitude/longitude for each university/school
mba_merged = mba_merged.join(mba_venues_sorted.set_index('University'), on='University')

mba_merged.head() # check the last columns!

,BusinessSchool,University,Location,USN2019,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Wharton School,University of Pennsylvania,"Pennsylvania, Philadelphia",1,39.949338,-75.189644,2,Coffee Shop,Park,Café,Restaurant,Food Truck,Bakery,American Restaurant,Plaza,Convenience Store,Italian Restaurant
1,Stanford Graduate School of Business,Stanford University,"California, Stanford",2,37.431314,-122.169365,2,Ice Cream Shop,Café,Gym / Fitness Center,Grocery Store,Art Museum,Pizza Place,Park,Cupcake Shop,Bookstore,New American Restaurant
2,Harvard Business School,Harvard University,"Massachusetts, Boston",3,42.367909,-71.126782,2,Café,Park,Pizza Place,American Restaurant,Bookstore,New American Restaurant,Plaza,Record Shop,Gastropub,Burger Joint
3,MIT Sloan School of Management,Massachusetts Institute of Technology,"Massachusetts, Cambridge",3,42.358396,-71.095678,2,Bakery,American Restaurant,Seafood Restaurant,Clothing Store,Cocktail Bar,Ice Cream Shop,Vegetarian / Vegan Restaurant,Bookstore,Sandwich Place,Café
4,Booth School of Business,University of Chicago,"Illinois, Chicago",3,41.791351,-87.600843,2,Park,Coffee Shop,Grocery Store,Café,Bookstore,Science Museum,History Museum,Art Gallery,Restaurant,Vegetarian / Vegan Restaurant


In [47]:
mba_merged

,BusinessSchool,University,Location,USN2019,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Wharton School,University of Pennsylvania,"Pennsylvania, Philadelphia",1,39.949338,-75.189644,2,Coffee Shop,Park,Café,Restaurant,Food Truck,Bakery,American Restaurant,Plaza,Convenience Store,Italian Restaurant
1,Stanford Graduate School of Business,Stanford University,"California, Stanford",2,37.431314,-122.169365,2,Ice Cream Shop,Café,Gym / Fitness Center,Grocery Store,Art Museum,Pizza Place,Park,Cupcake Shop,Bookstore,New American Restaurant
2,Harvard Business School,Harvard University,"Massachusetts, Boston",3,42.367909,-71.126782,2,Café,Park,Pizza Place,American Restaurant,Bookstore,New American Restaurant,Plaza,Record Shop,Gastropub,Burger Joint
3,MIT Sloan School of Management,Massachusetts Institute of Technology,"Massachusetts, Cambridge",3,42.358396,-71.095678,2,Bakery,American Restaurant,Seafood Restaurant,Clothing Store,Cocktail Bar,Ice Cream Shop,Vegetarian / Vegan Restaurant,Bookstore,Sandwich Place,Café
4,Booth School of Business,University of Chicago,"Illinois, Chicago",3,41.791351,-87.600843,2,Park,Coffee Shop,Grocery Store,Café,Bookstore,Science Museum,History Museum,Art Gallery,Restaurant,Vegetarian / Vegan Restaurant
5,Kellogg School of Management,Northwestern University,"Illinois, Evanston",6,42.055116,-87.675811,2,Coffee Shop,Pizza Place,Beach,Gym,Grocery Store,American Restaurant,Thai Restaurant,Bakery,Mexican Restaurant,Sushi Restaurant
6,Columbia Business School,Columbia University,"New York, New York City",6,40.807949,-73.961797,2,Park,Coffee Shop,Italian Restaurant,American Restaurant,Seafood Restaurant,Bakery,Grocery Store,Bookstore,Chinese Restaurant,Bagel Shop
7,Haas School of Business,"University of California, Berkeley","California, Berkeley",6,37.870946,-122.266399,2,Japanese Restaurant,Café,Yoga Studio,Bubble Tea Shop,American Restaurant,Pizza Place,Ice Cream Shop,Bookstore,Chinese Restaurant,Coffee Shop
8,Yale School of Management,Yale University,"Connecticut, New Haven",9,41.257131,-72.989670,1,Hotel,Sandwich Place,Donut Shop,Bank,Burger Joint,Gym / Fitness Center,Indian Restaurant,Mexican Restaurant,Miscellaneous Shop,Mobile Phone Shop
9,Ross School of Business,University of Michigan,"Michigan, Ann Arbor",10,42.294214,-83.710039,1,Coffee Shop,Park,Shipping Store,Pizza Place,Chinese Restaurant,Sandwich Place,Sushi Restaurant,Korean Restaurant,Supermarket,Food & Drink Shop


Then, the location map was colored according to the clusters.

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mba_merged['Latitude'], mba_merged['Longitude'], mba_merged['University'], mba_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Analyze the clusters

To gain inghts of each cluster, the top five venues in each cluster were analyzed.

First, the cluster labels were added to the one hot encoding.

In [49]:
mba_grouped2 = pd.merge(mba_grouped,
                        mba_merged[['University', 'Cluster Labels']],
                        on='University')
mba_grouped2.head()

,University,ATM,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit,Cluster Labels
0,Arizona State University,0.00,0.0,0.00,0.03,0.0,0.0,0.0,0.0,0.0,...,0.00,0.01,0.00,0.0,0.01,0.00,0.01,0.0,0.0,4
1,Boston College,0.00,0.0,0.00,0.03,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.0,1
2,Brigham Young University,0.02,0.0,0.00,0.01,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,1
3,Carnegie Mellon University,0.00,0.0,0.00,0.03,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.00,0.0,0.00,0.01,0.01,0.0,0.0,2
4,Columbia University,0.00,0.0,0.02,0.04,0.0,0.0,0.0,0.0,0.0,...,0.01,0.01,0.01,0.0,0.00,0.00,0.02,0.0,0.0,2


And then, a new dataframe was created by grouping by clusters.

In [50]:
cluster_grouped = mba_grouped2.groupby('Cluster Labels').mean().reset_index()
cluster_grouped.head()

,Cluster Labels,ATM,Accessories Store,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,0.002000,0.000000,0.000000,0.026138,0.000000,0.000000,0.000000,0.000000,0.001429,...,0.000000,0.000000,0.001010,0.001000,0.000000,0.000000,0.002109,0.004964,0.001493,0.000000
2,2,0.000000,0.000556,0.001111,0.029629,0.000556,0.000000,0.000000,0.000000,0.000556,...,0.000000,0.001111,0.005000,0.005710,0.000000,0.000556,0.003889,0.011763,0.001667,0.003333
3,3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,4,0.000526,0.000526,0.000526,0.030131,0.000526,0.000526,0.000526,0.002105,0.000526,...,0.000526,0.001696,0.001579,0.002349,0.000526,0.006880,0.003158,0.008096,0.000000,0.000526


Finally, the top five venues in each cluster were printed out.

In [51]:
num_top_venues = 5

for clu in cluster_grouped['Cluster Labels']:
    print('---- Cluster #', clu, '----')
    temp = cluster_grouped[cluster_grouped['Cluster Labels'] == clu].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Cluster # 0 ----
            venue  freq
0             Bar  0.08
1     Pizza Place  0.07
2            Café  0.07
3  Sandwich Place  0.07
4            Park  0.05


---- Cluster # 1 ----
                 venue  freq
0       Sandwich Place  0.05
1          Coffee Shop  0.04
2          Pizza Place  0.04
3  American Restaurant  0.03
4   Mexican Restaurant  0.03


---- Cluster # 2 ----
                 venue  freq
0          Coffee Shop  0.05
1  American Restaurant  0.03
2                 Café  0.03
3                 Park  0.03
4                Hotel  0.02


---- Cluster # 3 ----
        venue  freq
0  Food Truck  0.12
1     Stables  0.12
2       Hotel  0.06
3  Art Museum  0.06
4        Café  0.06


---- Cluster # 4 ----
                 venue  freq
0          Coffee Shop  0.05
1       Sandwich Place  0.04
2                  Bar  0.04
3          Pizza Place  0.04
4  American Restaurant  0.03




As a results, information on the top five venues in each cluster were obtained. My analysis ends here.